In [127]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

['creditcard.csv']


In [128]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE

In [129]:
data = pd.read_csv("../input/creditcard.csv")

## **Exploring the Data**


In [130]:
data.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


Data is PCA transformation, for maintaing confidentiality
Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'.

In [131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

Checking for Null.. Seems no missing valuesbelow :)

In [132]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

### Preprocessing of data

In [133]:
data["Amount"] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))

#Remove Time Colm
data = data.drop(['Time'], axis =1)

In [134]:
print("Non fradulent transaction : ", data[data.Class ==0 ].Class.count())
print("Fradulent transaction : ", data[data.Class ==1 ].Class.count())

Non fradulent transaction :  284315
Fradulent transaction :  492


## Modelling template for Random Forest and Neural network 

In [135]:
def RFClassifier(x_train,y_train,x_test,y_test):
    rf_model = RandomForestClassifier(n_estimators = 100)
    rf_model.fit(x_train,y_train)
    y_pred = rf_model.predict(x_test)
    print("Random Forest score:", rf_model.score(x_test,y_test))
    rf_cmatrics = confusion_matrix(y_test,y_pred.round())
    print("Random Forest Confusion matrix: \n", rf_cmatrics)

In [136]:
def KerasClass(x_train,y_train,x_test,y_test):
    model = Sequential([
        Dense(units =16, input_dim = 29 ,activation = 'relu'),
        Dense(units =24,activation = 'relu'),
        Dropout(0.5),
        Dense(20, activation = 'relu'),
        Dense(24 , activation = 'relu'),
        Dense(1, activation = 'sigmoid'),

    ])
    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.fit(x_train,y_train, batch_size = 15, epochs = 10)
    score = model.evaluate(x_test, y_test)
    print("TensorFlow score: ", score)
    y_pred = model.predict(x_test)
    y_test = pd.DataFrame(y_test)    
    cmatrics = confusion_matrix(y_test,y_pred.round())
    print("Tensorflow Confusion Matrix: \n" , cmatrics)

Highly imabalnce dataset..lets us try SMOTE and undersampling 

In [137]:
X = data.iloc[:, data.columns!='Class']
y = data.iloc[:, data.columns == 'Class']

In [138]:
#understampling
fraud_index = np.array(data[data.Class == 1].index)
no_rec_fraud = len(fraud_index)
print("number of Fradulent transaction :", no_rec_fraud)

nonfraud_index = np.array(data[data.Class == 0].index)

random_nomal_index = np.random.choice(nonfraud_index, no_rec_fraud, replace = False)
random_nomal_index = np.array(random_nomal_index)
print("number of non-Fradulent transaction by undersampling:",len(random_nomal_index))

number of Fradulent transaction : 492
number of non-Fradulent transaction by undersampling: 492


In [139]:
#combine Fradulent  and non-Fradulent trasaction.
under_sample = np.concatenate([fraud_index, random_nomal_index])
print("Total number of transaction by undersampling: ", len(under_sample))

Total number of transaction by undersampling:  984


In [140]:
under_sample_data = data.iloc[under_sample,:]

In [141]:
x_undersample = under_sample_data.iloc[:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.iloc[:, under_sample_data.columns == 'Class']

Train test split, 70:30 combination

In [142]:
x_train, x_test, y_train, y_test = train_test_split(x_undersample, y_undersample, test_size = 0.3)

In [143]:
RFClassifier(x_train,y_train,x_test,y_test)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Random Forest score: 0.956081081081081
Random Forest Confusion matrix: 
 [[147   4]
 [  9 136]]


In [144]:
KerasClass(x_train,y_train,x_test,y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 16)                480       
_________________________________________________________________
dense_62 (Dense)             (None, 24)                408       
_________________________________________________________________
dropout_13 (Dropout)         (None, 24)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 20)                500       
_________________________________________________________________
dense_64 (Dense)             (None, 24)                504       
_________________________________________________________________
dense_65 (Dense)             (None, 1)                 25        
Total params: 1,917
Trainable params: 1,917
Non-trainable params: 0
_________________________________________________________________
Epoch 1/

### Lets try with oversampling SMOTE

In [145]:
x_smote, y_smote = SMOTE().fit_sample(X, y.values.ravel())
x_smote = pd.DataFrame(x_smote)
y_smote = pd.DataFrame(y_smote)
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size = 0.3)

In [ ]:
#RFClassifier(x_train,y_train,x_test,y_test)

In [146]:
KerasClass(x_train,y_train,x_test,y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 16)                480       
_________________________________________________________________
dense_67 (Dense)             (None, 24)                408       
_________________________________________________________________
dropout_14 (Dropout)         (None, 24)                0         
_________________________________________________________________
dense_68 (Dense)             (None, 20)                500       
_________________________________________________________________
dense_69 (Dense)             (None, 24)                504       
_________________________________________________________________
dense_70 (Dense)             (None, 1)                 25        
Total params: 1,917
Trainable params: 1,917
Non-trainable params: 0
_________________________________________________________________
Epoch 1/